# Colombians migration data analysis

In [ ]:
import json
import requests
import pandas as pd
from sqlalchemy import create_engine

Primero, cargamos el link del dataset en formato json con una cantidad de datos igual a 200.000 ya que al cargar tantos datos podria demorar mas tiempo la carga de los mismos, las consultas y la visualizacion de los datos, tambien, lo cargamos desde el link porque al descargar el archivo, no podria subirlo a github por su tamaño.

In [2]:
# 1. Cargar config.json
with open("../config.json", "r") as f:
    config = json.load(f)

# 2. Extraer el token
token = config["api_keys"]["soda_token"]

# 3. Consumir la API 
url = "https://www.datos.gov.co/resource/y399-rzwf.json"

headers = {"X-App-Token": token}
params = {"$limit": 200000}

response = requests.get(url, headers=headers, params=params)
df = pd.DataFrame(response.json())

df.head()

,pa_s,c_digo_iso_pa_s,ciudad_de_residencia,oficina_de_registro,grupo_edad,edad_a_os,rea_conocimiento,sub_area_conocimiento,nivel_acad_mico,estado_civil,g_nero,etnia_de_la_persona,estatura_cm,ciudad_de_nacimiento,localizaci_n,fecha_de_registro,cantidad_de_personas
0,CHILE,CHL,TARAPACA/IQUIQUE,C. ANTOFAGASTA,ADULTO,45,NO INDICA,NO INDICA,BACHILLERATO,SOLTERO,FEMENINO,SIN ETNIA REGISTRADA,165,COLOMBIA/VALLE DEL CAUCA/BUENAVENTURA,"{'latitude': '-35.675147', 'longitude': '-71.5...",2015-08,2
1,VENEZUELA,VEN,CARABOBO/BEJUMA,C. VALENCIA VEN,ADULTO MAYOR,68,NINGUNA,NINGUNA,NO INDICA,CASADO,MASCULINO,OTRO,-1,COLOMBIA/SANTANDER/BUCARAMANGA,"{'latitude': '6.42375', 'longitude': '-66.5897...",2017-09,1
2,ITALIA,ITA,LOMBARDIA/BRESSANA BOTTARONE,C. MILAN,ADULTO,38,NINGUNA,NINGUNA,NO INDICA,SOLTERO,FEMENINO,OTRO,-1,COLOMBIA/ANTIOQUIA/YARUMAL,"{'latitude': '41.87194', 'longitude': '12.5673...",2022-04,1
3,CHILE,CHL,COQUIMBO/OVALLE,C. SANTIAGO,ADULTO,32,NINGUNA,NINGUNA,NO INDICA,CASADO,FEMENINO,NINGUNA,-1,COLOMBIA/VALLE DEL CAUCA/CALI,"{'latitude': '-35.675147', 'longitude': '-71.5...",2021-04,1
4,ESPA�A,ESP,CANTABRIA/SANTANDER,C. BILBAO,ADULTO,45,NINGUNA,NINGUNA,BACHILLERATO,SOLTERO,FEMENINO,OTRO,-1,COLOMBIA/HUILA/NEIVA,"{'latitude': '40.463667', 'longitude': '-3.749...",2023-07,1


Ahora, copiaremos el dataset a una base de datos PostgreSQL para realizar limpieza, transformación y análisis de datos para luego seguir con otros análisis exploratorios y visualizaciones más avanzadas con Python.

**Nota**: Corre esta celda de codigo una unica vez para subir los datos a PostgreSQL, ya que si la corres varias veces, se sobreescribiran los datos en la tabla "registro_aeropuerto" y si tienes algun tipo de dato modificado o agregado, se perdera.

In [3]:
# 1. Cargar config.json
with open('../config.json') as f:
    config = json.load(f)

# 2. Extraer los datos de mi servidor de PostgreSQL
db = config["database"]

# 3. Convertir diccionario a JSON en string
for col in df.columns:
    df[col] = df[col].apply(lambda x: json.dumps(x) if isinstance(x, dict) else x)

# 4. Conectar a PostgreSQL
engine = create_engine(
    f"postgresql://{db['user']}:{db['password']}@{db['host']}:{db['port']}/{db['dbname']}",
    connect_args={'options': '-c client_encoding=utf8'}
)
# 5. Subir DataFrame a PostgreSQL
df.to_sql('registro_aeropuerto', engine, if_exists='replace', index=False)
engine.dispose()

print('El dataset ha cargado exitosamente en la base de datos ✅')

El dataset ha cargado exitosamente en la base de datos ✅
